In [72]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [15]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

/var/folders/9q/dsjswm2x2q9_rxvxgvcvbqk00000gn/T/ipykernel_20848/3125191635.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [16]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [17]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [62]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import re
from razdel import tokenize
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger
from collections import Counter

import matplotlib.pyplot as plt

In [63]:
import nltk
from nltk import collocations 
nltk.download('genesis')

[nltk_data] Downloading package genesis to /Users/anna/nltk_data...
[nltk_data]   Package genesis is already up-to-date!


True

In [64]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/anna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [68]:
smr = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
dict_ = sorted(Counter(smr).items(), key=lambda x: -x[1])

In [69]:
high = set()
med = set()
low = set()
max_ = 10000 
min_ = 1000

In [71]:
for i in dict_:
    if i[1] > max_:
        high.add(i[0])
    elif i[1] < min_:
        low.add(i[0])
    else:
        med.add(i[0])
print(len(high),len(med),len(low))  

29 226 350868


In [89]:
stop = stopwords.words('russian') + list(punctuation) + list(med) + list(low)

In [90]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.80      0.58      0.67     38675
    positive       0.43      0.68      0.53     18034

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.68      0.61      0.62     56709



In [91]:
stop = stopwords.words('russian') + list(punctuation) + list(med)

In [92]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.75      0.76      0.75     27862
    positive       0.76      0.76      0.76     28847

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [93]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28002
    positive       1.00      1.00      1.00     28707

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [95]:
stop = stopwords.words('russian') + list(punctuation)

In [96]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.77      0.78      0.77     27653
    positive       0.78      0.77      0.78     29056

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [116]:
score_punc = pd.DataFrame(columns=['punc', 'acc'])
for i in list(punctuation):
    pred = ['positive' if i in tweet else 'negative' for tweet in x_test]
    score_punc = score_punc.append({'punc': i, 'acc': accuracy_score(pred, y_test)}, ignore_index=True)

In [119]:
score_punc.sort_values(by='acc', ascending=False).head(5)

,punc,acc
8,),0.913100
21,@,0.566154
15,:,0.544940
14,/,0.544340
26,_,0.518330


Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [122]:
from sklearn.feature_extraction.text import HashingVectorizer

In [124]:
vectorizer = HashingVectorizer(n_features=2**4,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.50      0.55      0.52     25911
    positive       0.59      0.55      0.57     30798

    accuracy                           0.55     56709
   macro avg       0.55      0.55      0.55     56709
weighted avg       0.55      0.55      0.55     56709



In [125]:
vectorizer = HashingVectorizer(n_features=2**8,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.60      0.61      0.61     27333
    positive       0.63      0.62      0.62     29376

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61      0.61      0.61     56709



In [126]:
vectorizer = HashingVectorizer(n_features=2**16,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.72      0.75      0.73     26887
    positive       0.76      0.74      0.75     29822

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709

